# api 크롤링

기존의 크롤링 방식은 

1. 내가 직접 홈페이지에 브라우저를 켜서 접근하거나(selenium) 

혹은 

2. 파이썬 자체적으로 호출을 요청해서(requests)

데이터를 가지고 오는 방식이었습니다.

이 방식은 서버에 부하가 많이 갈 뿐더러 나에게 필요없는 데이터까지 한 번에 호출해서 가져오는 문제가 있습니다.

따라서 서비스 제공자측에서는 서버 부하를 줄이고, 사용자에게 맞춤형을 데이터를 제공하기 위해 api서버를 운영합니다.

api서버는 인가된 데이터만을 개발자에게 넘겨서 서버도 안정적으로 유지하며(횟수 제한이 있는 사이트도 많습니다. 라이엇 데이터는 1초에 5회, 2분에 100회로 제한됩니다.) 

필요없는 동영상자료나 그림자료를 호출하지 않으므로 트래픽을 줄일 수 있습니다.



api 서버 접근시 보통 urllib.request 를 이용하게 됩니다.

In [1]:
# 사이트에 자료 요청
import urllib.request 
# json 데이터 핸들링
import json 
# DataFrame 자료형 활용
import pandas as pd 
# json 데이터를 pandas DataFrame으로 변환
from pandas.io.json import json_normalize 

# 영진위 api신청

http://www.kobis.or.kr/kobisopenapi/homepg/main/main.do

1. 접속 후 가입 및 로그인

2. 키발급 탭에서 키 발급하기

3. 요청 api종류 보고 요청양식 및 데이터 확인하기

In [2]:
api_key = "f5eef3421c602c6cb7ea224104795888" # 발급받은 키만 복붙하세요
request_date = "20220622" # 조회날짜를  xxxxyydd 20220623 형식으로 넣어주세요
url = "http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s" % (api_key, request_date)
print(url)

http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=f5eef3421c602c6cb7ea224104795888&targetDt=20220622


# json데이터를 팬더스 데이터프레임으로 변환

api 데이터는 보통 json(JavaScript Object Notation) 타입으로 받아집니다.

쉽게 말하면 자바스크립트 데이터를 전달하기 좋게 설정한 자료형인데

파이썬의 딕셔너리와 거의 같다고 보시면 됩니다.

따라서 json 데이터를 팬더스 데이터로 변환한다는 것은 사실상

딕셔너리 데이터를 팬더스 데이터프레임으로 변환하는것입니다.

In [3]:
# 위의 url 변수를 이용해 데이터를 요청합니다.
r = urllib.request.urlopen(url)

# 요청 페이지의 결과 데이터를 파이썬 내부 데이터로 바꿉니다.
json_raw = r.read()

In [4]:
# b'~~~~~~~'로 시작하는 이 데이터는 파이썬에서 byte자료형으로 처리되고
# 데이터프레임으로는 바로 변환이 불가능합니다.
json_raw

b'{"boxOfficeResult":{"boxofficeType":"\xec\x9d\xbc\xeb\xb3\x84 \xeb\xb0\x95\xec\x8a\xa4\xec\x98\xa4\xed\x94\xbc\xec\x8a\xa4","showRange":"20220622~20220622","dailyBoxOfficeList":[{"rnum":"1","rank":"1","rankInten":"0","rankOldAndNew":"NEW","movieCd":"20194376","movieNm":"\xed\x83\x91\xea\xb1\xb4: \xeb\xa7\xa4\xeb\xb2\x84\xeb\xa6\xad","openDt":"2022-06-22","salesAmt":"1976138530","salesShare":"54.3","salesInten":"1976138530","salesChange":"100","salesAcc":"2717027360","audiCnt":"188320","audiInten":"188320","audiChange":"100","audiAcc":"253024","scrnCnt":"1975","showCnt":"9158"},{"rnum":"2","rank":"2","rankInten":"-1","rankOldAndNew":"OLD","movieCd":"20224882","movieNm":"\xeb\xa7\x88\xeb\x85\x80(\xe9\xad\x94\xe5\xa5\xb3) Part2. The Other One","openDt":"2022-06-15","salesAmt":"836530290","salesShare":"23.0","salesInten":"-253595090","salesChange":"-23.3","salesAcc":"18226709650","audiCnt":"82697","audiInten":"-25485","audiChange":"-23.6","audiAcc":"1769487","scrnCnt":"977","showCnt":"35

In [5]:
# 바이트자료형임을 확인
type(json_raw)

bytes

In [6]:
# 따라서 utf-8 형식으로 고쳐줘야 합니다. .decode()를 이용합니다.
# encode => 우리가 쓰는 문자를 컴퓨터가 쓰는 형태로 변환
# decode => 컴퓨터가 쓰는 형태의 문자를 우리가 쓰는 형태로 변환
json_utf8 = json_raw.decode("utf-8")

In [7]:
# 딕셔너리 구조를 유지하고 있는 문자로 변환됨
type(json_utf8)

str

In [8]:
# json.loads(딕셔너리 형태의 문자열 자료)
# 딕셔너리 형태로 적힌 문자열 자료를 딕셔너리형으로 변환
json_complete = json.loads(json_utf8)

In [9]:
# 딕셔너리(json) 변환 완료 확인
type(json_complete)

dict

In [10]:
# json_normalize(딕셔너리 자료)
# 딕셔너리 자료를 데이터프레임 형태로 변환해줌.
pd.json_normalize(json_complete)

,boxOfficeResult.boxofficeType,boxOfficeResult.showRange,boxOfficeResult.dailyBoxOfficeList
0,일별 박스오피스,20220622~20220622,"[{'rnum': '1', 'rank': '1', 'rankInten': '0', ..."


In [11]:
# boxOfficeResult.dailyBoxOfficeList 컬럼 내부에 딕셔너리가 여럿 묶인 리스트가 보이므로
# boxOfficeResult.dailyBoxOfficeList 컬럼을 타겟으로 데이터프레임화 합니다.
box_result = pd.json_normalize(json_complete['boxOfficeResult']['dailyBoxOfficeList'])

In [12]:
# json_normalize()는 pd.DataFrame() 으로 대체 가능합니다.
pd.DataFrame(json_complete)

,boxOfficeResult
boxofficeType,일별 박스오피스
dailyBoxOfficeList,"[{'rnum': '1', 'rank': '1', 'rankInten': '0', ..."
showRange,20220622~20220622


In [13]:
import time
import datetime # 파이썬 날짜 자료형

In [14]:
start_datetime = datetime.datetime(2021, 6, 22) # 년 월 일 순으로 넣어줍니다.
# 형식 바꾸기
target_date = start_datetime.strftime('%Y%m%d') # 영진위 형식으로 변경
print(target_date)
# 날짜 하루 올리기
next_datetime = start_datetime + datetime.timedelta(days=1)
next_datetime

20210622


datetime.datetime(2021, 6, 23, 0, 0)

In [15]:
# target_dates = ['20220620','20220621','20220622']
# 날짜는 시작날짜와 + 며칠 을 처리할지 결정할 수 있습니다.
# 1년치 데이터를 얻은 다음 to_csv나 to_excel로 저장해주세요.
start_datetime = datetime.datetime(2021, 6, 22) # 년 월 일 순으로 넣어줍니다.

# 반복문과 append를 이용해서 3일치 데이터를 요청해서 합쳐보세요.
# target_date 컬럼에 조회 날짜를 추가해주세요.
# 빈 데이터프레임을 하나 생성해놓고(pd.DataFrame())
merged_data = pd.DataFrame()
for date in range(365):
    target_date = start_datetime.strftime('%Y%m%d')
    url = "http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s" % (api_key, target_date)
    # 위의 url 변수를 이용해 데이터를 요청합니다.
    r = urllib.request.urlopen(url)

    # 요청 페이지의 결과 데이터를 파이썬 내부 데이터로 바꿉니다.
    json_raw = r.read()
    json_utf8 = json_raw.decode("utf-8")
    json_complete = json.loads(json_utf8)
    box_result = pd.json_normalize(json_complete['boxOfficeResult']['dailyBoxOfficeList'])
    # 얻어온 데이터에 날짜 컬럼 추가
    box_result['target_date'] = target_date
    # 날짜별 데이터프레임을 하나로 통합
    merged_data = pd.concat([merged_data, box_result])
    start_datetime = start_datetime + datetime.timedelta(days=1)
    print("%s일차 수집 완료" % date)
    
merged_data

0일차 수집 완료
1일차 수집 완료
2일차 수집 완료
3일차 수집 완료
4일차 수집 완료
5일차 수집 완료
6일차 수집 완료
7일차 수집 완료
8일차 수집 완료
9일차 수집 완료
10일차 수집 완료
11일차 수집 완료
12일차 수집 완료
13일차 수집 완료
14일차 수집 완료
15일차 수집 완료
16일차 수집 완료
17일차 수집 완료
18일차 수집 완료
19일차 수집 완료
20일차 수집 완료
21일차 수집 완료
22일차 수집 완료
23일차 수집 완료
24일차 수집 완료
25일차 수집 완료
26일차 수집 완료
27일차 수집 완료
28일차 수집 완료
29일차 수집 완료
30일차 수집 완료
31일차 수집 완료
32일차 수집 완료
33일차 수집 완료
34일차 수집 완료
35일차 수집 완료
36일차 수집 완료
37일차 수집 완료
38일차 수집 완료
39일차 수집 완료
40일차 수집 완료
41일차 수집 완료
42일차 수집 완료
43일차 수집 완료
44일차 수집 완료
45일차 수집 완료
46일차 수집 완료
47일차 수집 완료
48일차 수집 완료
49일차 수집 완료
50일차 수집 완료
51일차 수집 완료
52일차 수집 완료
53일차 수집 완료
54일차 수집 완료
55일차 수집 완료
56일차 수집 완료
57일차 수집 완료
58일차 수집 완료
59일차 수집 완료
60일차 수집 완료
61일차 수집 완료
62일차 수집 완료
63일차 수집 완료
64일차 수집 완료
65일차 수집 완료
66일차 수집 완료
67일차 수집 완료
68일차 수집 완료
69일차 수집 완료
70일차 수집 완료
71일차 수집 완료
72일차 수집 완료
73일차 수집 완료
74일차 수집 완료
75일차 수집 완료
76일차 수집 완료
77일차 수집 완료
78일차 수집 완료
79일차 수집 완료
80일차 수집 완료
81일차 수집 완료
82일차 수집 완료
83일차 수집 완료
84일차 수집 완료
85일차 수집 완료
86일차 수집 완료
87일차 수집 완료
88일차 수집 완료
89일차 수집 완료
90일차 수집 완료
91일차 수집 완

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,target_date
0,1,1,0,OLD,20204261,콰이어트 플레이스 2,2021-06-16,294889050,31.5,-30865510,-9.5,4421657460,30612,-3293,-9.7,444223,1018,3714,20210622
1,2,2,0,OLD,20216362,크루엘라,2021-05-26,251308720,26.8,2055160,0.8,12310491420,26091,229,0.9,1277523,800,2102,20210622
2,3,3,0,OLD,20216685,루카,2021-06-17,87755080,9.4,2066470,2.4,1395603040,9669,303,3.2,152715,620,1668,20210622
3,4,4,0,OLD,20217736,컨저링3: 악마가 시켰다,2021-06-03,88336660,9.4,-2132260,-2.4,7416374550,8693,-160,-1.8,738471,547,981,20210622
4,5,5,0,OLD,20218240,여고괴담 여섯번째 이야기 : 모교,2021-06-17,52326870,5.6,-4705000,-8.2,697801050,5725,-381,-6.2,77363,466,1307,20210622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,6,0,NEW,20209654,헤어질 결심,2022-06-29,51374000,2.1,51374000,100,51374000,5706,5706,100,5706,14,23,20220621
6,7,7,1,OLD,20135304,극장판 포켓몬스터DP: 기라티나와 하늘의 꽃다발 쉐이미,2022-06-01,7169000,0.3,2203800,44.4,5286590290,844,282,50.2,561229,77,88,20220621
7,8,8,1,OLD,20210843,실종,2022-06-15,7153500,0.3,1744500,32.3,68060800,749,208,38.4,6796,61,85,20220621
8,9,9,-2,OLD,20212855,닥터 스트레인지: 대혼돈의 멀티버스,2022-05-04,7209400,0.3,-376470,-5,62627623410,738,-3,-0.4,5882504,26,45,20220621


In [16]:
box_result.head()

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,target_date
0,1,1,0,OLD,20224882,마녀(魔女) Part2. The Other One,2022-06-15,1090125380,45.2,-126228690,-10.4,17390179360,108182,-12316,-10.2,1686790,1630,6629,20220621
1,2,2,0,OLD,20204548,범죄도시 2,2022-05-18,736323430,30.5,-57806350,-7.3,120084185570,73870,-5657,-7.1,11613691,1132,4569,20220621
2,3,3,0,OLD,20206257,브로커,2022-06-08,217715280,9.0,15074160,7.4,11519502600,22362,1540,7.4,1150668,665,1977,20220621
3,4,4,0,OLD,20223839,버즈 라이트이어,2022-06-15,134676940,5.6,6735280,5.3,2459282490,12407,1252,11.2,229329,626,1776,20220621
4,5,5,0,OLD,20206061,쥬라기 월드: 도미니언,2022-06-01,97234960,4.0,9478580,10.8,28756258110,9956,1255,14.4,2786272,429,783,20220621


In [17]:
# csv 혹은 엑셀로 뽑아주세요.
# 스크린당 평균 관객이 제일 많았던 영화를 뽑아주세요.
# 관객 총합 / 스크린 총합 = 스크린당 관객이 나옵니다.
# 컬럼 타입이 object이므로, 집계함수를 쓸 수 있도록 int로 컬럼 전체 자료형을 바꿔줘야 합니다.
merged_data['audiCnt'] = merged_data['audiCnt'].astype(int)
merged_data['scrnCnt'] = merged_data['scrnCnt'].astype(int)
merged_data['target_date'] = pd.to_datetime(merged_data['target_date'])

In [18]:
merged_data['target_date']

0   2021-06-22
1   2021-06-22
2   2021-06-22
3   2021-06-22
4   2021-06-22
       ...    
5   2022-06-21
6   2022-06-21
7   2022-06-21
8   2022-06-21
9   2022-06-21
Name: target_date, Length: 3650, dtype: datetime64[ns]

In [19]:
mean_data = merged_data.groupby('movieNm')[['audiCnt', 'scrnCnt']].sum()

In [20]:
mean_data

,audiCnt,scrnCnt
movieNm,,
007 노 타임 투 다이,1222296,36288
1984 최동원,4429,494
355,37910,3440
BIAF2021 부천국제애니메이션페스티벌 기획상영展,539,3
BIAF2021 한국 단편 경쟁 B,3582,9
...,...,...
호빗: 스마우그의 폐허,1824,70
호스트: 접속금지,13218,965
화양연화,3858,88


In [21]:
mean_data['result'] = mean_data['audiCnt'] / mean_data['scrnCnt']

In [22]:
db_insert_data = mean_data[['audiCnt', 'scrnCnt', 'result']].sort_values(by='result', ascending=False)

In [23]:
# csv로 빼내기
merged_data.to_excel('영진위.xlsx', encoding="utf-8")

# 가져온 데이터 mysql에 연동하기

- import pymysql
- 을 활용해서 연동할 수 있습니다.

- 임포트가 되지 않는다면 다시 설치를 해야합니다.
1. Anaconda Navigator에서 좌측 탭의 enviroments를 누릅니다.
2. Open terminal을 이용해 cmd창같은 창을 켭니다.(터미널창이라고 부릅니다.)
3. pip install pymysql 을 입력하고 실행합니다.

### 오라클 연동

위와 같은데 3번에서
- pip install cx_Oracle 
- 을 대신 입력해서 설치하면 됩니다.


In [24]:
# pymysql 실행을 해야 DB와 연결할 수 있습니다.
import pymysql

In [25]:
# MySQL 연결에 필요한 정보를 입력합니다.
con = pymysql.connect(host='localhost', user='root', password='mysql', db='pyprac1', charset='utf8',
                     cursorclass=pymysql.cursors.DictCursor) # 컬럼명까지 받아올지 여부

In [26]:
# SQL구문을 받아주는 cursor 객체를 사용합니다.
cur = con.cursor()

In [27]:
# SQL구문은 문자로 cur.execute()를 실행해서 수행시킵니다.
cur.execute("SELECT * FROM test_tbl")

6

In [28]:
# execute가 실행되면, cur가 데이터를 저장하고 있는데 이걸 파이선 형식으로 뺴야합니다.
mysql_data = cur.fetchall()

In [29]:
mysql_data

[{'tno': 1, 'tname': 'a'},
 {'tno': 2, 'tname': 'b'},
 {'tno': 3, 'tname': 'c'},
 {'tno': 4, 'tname': 'd'},
 {'tno': 5, 'tname': 'e'},
 {'tno': 6, 'tname': 'e'}]

In [30]:
# 데이터프레임 형태로 변환
pd.DataFrame(mysql_data)

,tno,tname
0,1,a
1,2,b
2,3,c
3,4,d
4,5,e
5,6,e


In [31]:
# 여러분들이 data 'e'를 넣어보세요.
cur.execute("INSERT INTO test_tbl VALUES (null, 'e')")

1

In [32]:
# 커서가 아닌 커넥션에 commit을 주는것에 주의해주세요.
con.commit()

In [33]:
# db_insert_data에는 movieNm이라는 로우가 있는데 컬럼으로 바꾸는게 DB에 넣을때 유리함
db_insert_data
# 전체 로우 데이터를 컬럼에 새로 집어넣어주기
db_insert_data['movieNm'] = db_insert_data.index

In [34]:
# reset_index로 로우 삭제하기
db_insert_data = db_insert_data.reset_index(drop=True)

In [35]:
db_insert_data.head()

,audiCnt,scrnCnt,result,movieNm
0,1392,1,1392.0,내일의 기억
1,1238,1,1238.0,파이프라인
2,2000,2,1000.0,비긴 어게인
3,2000,2,1000.0,어바웃 타임
4,710,1,710.0,더 스파이


In [36]:
#로우개수
len(db_insert_data)
# loc를 이용해 특정 로우 자료를, 그리고 다시 ['컬럼조회']를 통해 특정로우의 특정컬럼값을 조회할 수 있습니다.
db_insert_data.loc[1]['audiCnt']

1238

In [37]:
# 과제. MySQL이나 Oracle SQL에 현재 보고있는 db_insert데이터를
# 테이블 생성 및 반복문을 이용해서 다 DB에 넣어주시면 됩니다.

# 위 로직을 똑같이 오라클에서도 활용할 수 있습니다. 직접 해 보세요.

In [63]:
import cx_Oracle

In [64]:
# Oracle 연동
# 11버전은 localhost:1521/XE
# 18버전은 localhost:1521/XEPDB1
con_oracle = cx_Oracle.connect("mytest", "mytest", "localhost:1521/XE")

In [65]:
cur_oracle = con_oracle.cursor()

In [66]:
cur_oracle.execute("SELECT * FROM movie_tbl")

<cx_Oracle.Cursor on <cx_Oracle.Connection to mytest@localhost:1521/XE>>

In [67]:
cur_oracle.fetchall()

[('테스트영화', 1, 1, 1)]

In [68]:
db_insert_data.head()

,audiCnt,scrnCnt,result,movieNm
0,1392,1,1392.0,내일의 기억
1,1238,1,1238.0,파이프라인
2,2000,2,1000.0,비긴 어게인
3,2000,2,1000.0,어바웃 타임
4,710,1,710.0,더 스파이


In [69]:
for row_num in range(len(db_insert_data)):
    # movieNm 각 항목별로 반복문 돌려서 '가 하나 이상 발견되는 경우
    # 파이썬은 0이면 False, 나머지는 True이므로 count결과를 그대로 조건식에 넣을수 있음
    if(db_insert_data.loc[row_num]['movieNm'].count("'")):
        db_insert_data.loc[row_num]['movieNm'] = db_insert_data.loc[row_num]['movieNm'].replace("'", "")
        

In [70]:
db_insert_data['movieNm'][220] = db_insert_data.loc[220]['movieNm'].replace("'", "")

C:\Users\ict03-00\AppData\Local\Temp\ipykernel_17344\2693812650.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db_insert_data['movieNm'][220] = db_insert_data.loc[220]['movieNm'].replace("'", "")


In [71]:
db_insert_data.loc[220]['movieNm']

'뱅드림! 팝핀 드림!'

In [72]:
for row_num in range(len(db_insert_data)):
    movie_nm = db_insert_data.iloc[row_num]['movieNm']
    audi_cnt = db_insert_data.iloc[row_num]['audiCnt']
    scrn_cnt = db_insert_data.iloc[row_num]['scrnCnt']
    audi_mean = db_insert_data.iloc[row_num]['result']
    query = "INSERT INTO movie_tbl VALUES ('%s', %s, %s, %s)" % (movie_nm, audi_cnt, scrn_cnt, int(audi_mean))
    print(query)
    cur_oracle.execute(query)
    print("-----------------------------------------------")
con_oracle.commit()

INSERT INTO movie_tbl VALUES ('내일의 기억', 1392, 1, 1392)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('파이프라인', 1238, 1, 1238)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('비긴 어게인', 2000, 2, 1000)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('어바웃 타임', 2000, 2, 1000)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('더 스파이', 710, 1, 710)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('BIFAN2021 판타스틱 단편 걸작선 1', 20000, 37, 540)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('라라랜드', 2022, 4, 505)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('헤어질 결심', 5706, 14, 407)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('BIAF2021 한국 단편 경쟁 B', 3582, 9, 398)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('지지', 334, 1, 334)
-

-----------------------------------------------
INSERT INTO movie_tbl VALUES ('레드 아워', 1000, 50, 20)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('청킹맨션', 1000, 50, 20)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('하빗', 1000, 50, 20)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('우연과 상상', 12002, 608, 19)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('이번엔 잘 되겠지', 1731, 88, 19)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('컨저링3: 악마가 시켰다', 64724, 3300, 19)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('공기살인', 151746, 7824, 19)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('몬스터 아카데미', 3859, 200, 19)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('귀멸의 칼날: 남매의 연', 58220, 3025, 19)
-----------------------------------------------
INSERT INTO

-----------------------------------------------
INSERT INTO movie_tbl VALUES ('사랑하고 사랑받고 차고 차이고', 1721, 169, 10)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('나이트 레이더스', 731, 72, 10)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('스페이스 독 3: 트로피컬 어드벤처', 3285, 327, 10)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('마이 뉴욕 다이어리', 3409, 342, 9)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('어부바', 20093, 2023, 9)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('말임씨를 부탁해', 12626, 1278, 9)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('킬링 카인드: 킬러의 수제자', 2167, 220, 9)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('라라와 크리스마스 요정', 2980, 308, 9)
-----------------------------------------------
INSERT INTO movie_tbl VALUES ('민스미트 작전', 29580, 3085, 9)
-------------------------------

In [62]:
cur_oracle.close()
con_oracle.close()